In [ ]:
import numpy as np
import cupy as cp
import h5py
from holotomocupy.holo import G, GT
from holotomocupy.shift import S, ST
from holotomocupy.recon_methods import multiPaganin
from holotomocupy.utils import *
from holotomocupy.proc import remove_outliers
import cv2
import xraylib
##!jupyter nbconvert --to script config_template.ipynb

# Init data sizes and parametes of the PXM of ID16A

In [ ]:
n = 512  # object size in each dimension
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
ndist = 4
distances = np.array([0.0029432,0.00306911,0.00357247,0.00461673])[:ndist] # [m]
magnification = 400
detector_pixelsize = 3.03751e-6
voxelsize = detector_pixelsize/magnification*2048/n  # object voxel size

ne = n+n//4
pad=n//8
show = True

path = f'/data/vnikitin/modeling/siemens{n}'


## Modeling data

In [ ]:
img = np.zeros((ne, ne, 3), np.uint8)
c=0.5
triangle = np.array([(ne//16/c, ne//2-ne//32), (ne//16/c, ne//2+ne//32), (ne//2-ne//128, ne//2)], np.float32)
star = img[:,:,0]*0
for i in range(0, 360, 15):
    img = np.zeros((ne, ne, 3), np.uint8)
    degree = i
    theta = degree * np.pi / 180
    rot_mat = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]], np.float32)    
    rotated = cv2.gemm(triangle-ne//2, rot_mat, 1, None, 1, flags=cv2.GEMM_2_T)+ne//2
    cv2.fillPoly(img, [np.int32(rotated)], (255, 0, 0))
    star+=img[:,:,0]
[x,y] = np.meshgrid(np.arange(-ne//2,ne//2),np.arange(-ne//2,ne//2))
x = x/ne*2
y = y/ne*2
# add holes in triangles
circ = (x**2+y**2>0.145)+(x**2+y**2<0.135)
circ *= (x**2+y**2>0.053)+(x**2+y**2<0.05)
circ *= (x**2+y**2>0.0085)+(x**2+y**2<0.008)
circ *= (x**2+y**2>0.52)+(x**2+y**2<0.5)

star = star*circ/255

# star[ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2]=0

v = np.arange(-ne//2,ne//2)/ne
[vx,vy] = np.meshgrid(v,v)
v = np.exp(-5*(vx**2+vy**2))
fu = np.fft.fftshift(np.fft.fftn(np.fft.fftshift(star)))
star = np.fft.fftshift(np.fft.ifftn(np.fft.fftshift(fu*v))).real

delta = 1-xraylib.Refractive_Index_Re('Au',energy,19.3)
beta = xraylib.Refractive_Index_Im('Au',energy,19.3)

thickness = 2e-6/voxelsize # siemens star thickness in pixels
# form Transmittance function
u = star*(-delta+1j*beta) # note -delta
Ru = u*thickness 
psi = np.exp(1j * Ru * voxelsize * 2 * np.pi / wavelength)[np.newaxis].astype('complex64')
fig, axs = plt.subplots(1, 2, figsize=(9, 4))
im=axs[0].imshow(np.abs(psi[0]),cmap='gray')
axs[0].set_title('amplitude')
fig.colorbar(im)
im=axs[1].imshow(np.angle(psi[0]),cmap='gray')
axs[1].set_title('phase')
fig.colorbar(im)

mshow_polar(psi[0,ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2],show)

from scipy.io import savemat
savemat('data.mat',{'psi': psi[0,ne//2-n//2:ne//2+n//2,ne//2-n//2:ne//2+n//2]})

In [ ]:
psi=cp.array(psi)
v = cp.ones(ne,dtype='float32')
fs = n//16

v[:fs] = cp.sin(cp.linspace(0,1,fs)*cp.pi/2)
v[-fs:] = cp.cos(cp.linspace(0,1,fs)*cp.pi/2)
v = cp.outer(v,v)
mshow(v,show)
mshow_polar(v*psi[0],show)


# Construct operators


In [ ]:
def Lop(psi):
    data = cp.zeros([1,ndist, ne, ne], dtype='complex64')
    for i in range(ndist):
        psir = cp.array(psi).copy()            
           
        # v = cp.ones(ne,dtype='float32')
        # v[:fs] = cp.sin(cp.linspace(0,1,fs)*cp.pi/2)
        # v[-fs:] = cp.cos(cp.linspace(0,1,fs)*cp.pi/2)
        # v = cp.outer(v,v)
        # psir *= v

        psir = G(psir, wavelength, voxelsize, distances[i],'symmetric')        

        data[:, i] = psir
    return data

def LTop(data):
    psi = cp.zeros([1, ne, ne], dtype='complex64')
    for j in range(ndist):
        datar = data[:, j].copy()
        datar = GT(datar, wavelength, voxelsize, distances[j],'symmetric')        

        # v = cp.ones(ne,dtype='float32')
        # v[:fs] = cp.sin(cp.linspace(0,1,fs)*cp.pi/2)
        # v[-fs:] = cp.cos(cp.linspace(0,1,fs)*cp.pi/2)
        # v = cp.outer(v,v)
        # datar *= v
        psi += datar
    return psi



def Cfop(e):
    res = e.copy()
    res[:,:,pad:ne-pad,pad:ne-pad] = 0
    return res

def CfTop(e):
    res = e.copy()
    res[:,:,pad:ne-pad,pad:ne-pad] = 0
    return res

def Cdop(psi):
    return psi[:,:,pad:-pad,pad:-pad]

def CdTop(psi):
    return cp.pad(psi,((0,0),(0,0),(pad,pad),(pad,pad)))




In [ ]:
psi = cp.array(psi)
data = np.abs(Cdop(Lop(psi)))**2
ref = np.abs(Lop((psi*0+1)))**2

for k in range(ndist):
    mshow_complex(data[0,k]+1j*(data[0,k]-data[0,0]),show)



In [ ]:
import os 
os.system(f"mkdir -p {path}")
# np.save(f'{path}/data',data.get())
# np.save(f'{path}/psi',psi.get())
